In [1]:
import pandas as pd
import numpy as np

In [32]:
cl_dt= pd.read_csv(r'C:\GH-Dataset\Classification_data.csv')

In [33]:
cl_dt.shape

(906, 1428)

In [36]:
cl_dt.sample(n=10,replace=False)

,Doc_ID,label,__future__,__main__,_thread,abc,argparse,array,ast,asyncio,...,export,kill,logs,pause,reload,remove,rename,restart,stats,top
140,Doc141,img,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
177,Doc178,img,2,0,0,0,5,13,0,0,...,0,1,0,0,0,1,0,0,0,0
544,Doc545,inf,1,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
258,Doc259,nlp,15,0,5,0,11,0,1,0,...,0,0,0,0,0,5,0,0,0,0
323,Doc324,nlp,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
903,Doc904,cloud,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,Doc37,img,1,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
277,Doc278,nlp,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150,Doc151,img,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
660,Doc661,inf,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test = train_test_split(cl_dt, test_size=0.30, random_state=42)

In [39]:
X_train=X_train.reset_index()

In [40]:
X_train=X_train.drop(['level_0'], axis=1)

In [41]:
X_test=X_test.reset_index()

In [42]:
X_test=X_test.drop(['level_0'], axis=1)

In [49]:
def list_token(doc , ack):
    temp1=pd.DataFrame(doc).transpose()
    temp2 = temp1.loc[:, (temp1 != 0).any(axis=0)]
    if ack==0:
        freq = temp2.drop(['Doc_ID','label'], axis=1)
    else:
        freq = temp2.drop(['label'], axis=1)
    list_words =freq.columns.tolist()
    return list_words

In [46]:
word_cl_dic={}
n = len(X_train)

In [51]:
for i in range(0,len(X_train)):
    dt=X_train.iloc[i,]
    lst_words=list_token(dt,0)
    cl=dt['label']
    for w in set(lst_words):
        word_cl_dic[w]=word_cl_dic.get(w,{})
        word_cl_dic[w][cl]=word_cl_dic[w].get(cl,0)
        word_cl_dic[w][cl]+=1

In [52]:
word_cat_dic

{'get': {'cloud': 51, 'ntw': 47, 'img': 18, 'inf': 56, 'nlp': 28},
 'environ': {'cloud': 18, 'img': 9, 'inf': 16, 'ntw': 14, 'nlp': 10},
 'error': {'ntw': 23, 'img': 4, 'inf': 20, 'cloud': 8, 'nlp': 9},
 'flush': {'ntw': 6, 'img': 6, 'nlp': 6, 'inf': 6, 'cloud': 6},
 'struct': {'ntw': 8, 'img': 1, 'inf': 4, 'cloud': 2},
 'split': {'ntw': 25, 'cloud': 22, 'img': 22, 'nlp': 36, 'inf': 24},
 'connect': {'ntw': 12, 'img': 7, 'inf': 9, 'nlp': 6, 'cloud': 1},
 'close': {'ntw': 26, 'img': 12, 'inf': 16, 'nlp': 16, 'cloud': 8},
 'sys': {'ntw': 29, 'cloud': 13, 'img': 23, 'inf': 20, 'nlp': 25},
 'select': {'ntw': 2, 'inf': 13, 'img': 2, 'cloud': 1, 'nlp': 1},
 'logging': {'ntw': 12, 'img': 12, 'nlp': 16, 'inf': 21, 'cloud': 16},
 'warning': {'ntw': 9, 'img': 11, 'nlp': 4, 'inf': 12, 'cloud': 16},
 'send': {'ntw': 15, 'img': 7, 'inf': 5, 'cloud': 2},
 'recv': {'ntw': 13, 'inf': 4, 'cloud': 1},
 'range': {'ntw': 26, 'img': 45, 'nlp': 31, 'inf': 23, 'cloud': 14},
 'socket': {'ntw': 17, 'cloud': 2,

In [69]:
word_features={}
k=100

In [65]:
num_doc_cl=X_train.groupby(['label'])['Doc_ID'].count()

In [66]:
num_doc_cl

label
cloud    141
img      146
inf      125
nlp       89
ntw      133
Name: Doc_ID, dtype: int64

In [67]:
import operator
from math import log

In [70]:
for w in word_cat_dic.keys():
    dic = word_cat_dic[w]
    n_kw=0
    for cl in dic.keys():
        n_kw+=dic[cl] ## number of documents in the training set where the word occurs
    for cl in dic.keys():
        word_features[cl]=word_features.get(cl,[])
        n_kw_cl=dic[cl]
        n_dev_cl=num_doc_cl.loc[cl]
        p_w=n_kw/n
        p_cl = n_dev_cl/n
        p_w_cl= n_kw_cl/n
        
        pmi=log((p_w_cl)/(p_w*p_cl))
        
        if len(word_features[cl])<k:
            word_features[cl].append((w,pmi))
            if len(word_features[cl])==k:
                word_features[cl].sort(key=operator.itemgetter(1),reverse=True)                
        else:
            if word_features[cl][k-1][1]<pmi:
                word_features[cl][k-1]= (w,pmi)
                word_features[cl].sort(key=operator.itemgetter(1),reverse=True)    

In [71]:
word_features

{'cloud': [('delete_function', 1.5032890640590577),
  ('describe_stacks', 1.5032890640590577),
  ('s3', 1.5032890640590577),
  ('create_stack', 1.5032890640590577),
  ('update_stack', 1.5032890640590577),
  ('delete_stack', 1.5032890640590577),
  ('tag_resource', 1.5032890640590577),
  ('list_tags_for_resource', 1.5032890640590577),
  ('untag_resource', 1.5032890640590577),
  ('delete_policy', 1.5032890640590577),
  ('get_waiter', 1.5032890640590577),
  ('validate_configuration', 1.5032890640590577),
  ('codebuild', 1.5032890640590577),
  ('rekognition', 1.5032890640590577),
  ('apigateway', 1.5032890640590577),
  ('xray', 1.5032890640590577),
  ('codepipeline', 1.5032890640590577),
  ('traits', 1.5032890640590577),
  ('delete_connection', 1.5032890640590577),
  ('post_to_connection', 1.5032890640590577),
  ('unit', 1.5032890640590577),
  ('get_finding', 1.5032890640590577),
  ('securityhub', 1.5032890640590577),
  ('list_tags', 1.5032890640590577),
  ('nullsession', 1.5032890640590577

In [72]:
word_list=[]
for cat in word_features.keys():
    word_features[cat]=dict(word_features[cat])
    for w in word_features[cat]:
        word_list.append(w)

In [98]:
#word_list

In [109]:
dt_per_cat=X_train.groupby(['label']).sum().reset_index()

In [110]:
#dt_per_cl

In [111]:
cat_word_count_dict={}
cat_word_dict={}

In [113]:
for j in range(0,len(dt_per_cat)):
    cat = dt_per_cat.iloc[j]['label']
    temp_per_cat=dt_per_cat.iloc[j,]
    list_word=list_token(temp_per_cat,1)
    list_words = [w for w in list_word if word_features[cat].get(w,-100000)!=-100000]
    #dt=temp_per_cat.drop(['label'])
    #sum_all=np.sum(dt)
    #cat_word_count_dict[cat]=sum_all
    #list_words=list_token_without_author(temp_per_cat)
    cat_word_count_dict[cat] = cat_word_count_dict.get(cat,0)
    cat_word_dict[cat] = cat_word_dict.get(cat,{})
    sum_all=0
    for w in set(list_words):
        cat_word_dict[cat][w] = cat_word_dict[cat].get(w,0)
        cat_word_dict[cat][w] = temp_per_cat[w]
        sum_all += temp_per_cat[w]
        
    cat_word_count_dict[cat]=sum_all

In [114]:
cl_word_count_dic

{'cloud': 1020, 'img': 3769, 'inf': 1508, 'nlp': 1851, 'ntw': 1281}

In [115]:
#cl_word_dic

In [116]:
vocab_length=0            
for cl in cl_word_dic.keys():
    length = len(cl_word_dic[cat])
    vocab_length+=length

In [117]:
li_results=[]

In [120]:
for i in range(0,len(X_test)):
    
    minimum_neg_log_prob=1000000000
    min_category=''
    dt=X_test.iloc[i,]
    list_word=list_token(dt,0)
    list_words = [w for w in list_word if w in word_list]
    cat_dev=dt['label']
    dev_name = dt['Doc_ID']
    
    for cat in  cat_word_count_dict:
        neg_log_prob=-log(num_doc_cl.loc[cat]/n)
        word_dict = cat_word_dict[cat]
        count_cat = cat_word_count_dict[cat]
        for w in list_words:
            count_word_train=word_dict.get(w,0)
            ratio = (count_word_train+1)/(count_cat+vocab_length)
            neg_log_prob-=log(ratio)
            
        if minimum_neg_log_prob>neg_log_prob:
            min_category=cat
            minimum_neg_log_prob=neg_log_prob
            
    li_results.append((dev_name,min_category,cat_dev))

In [105]:
list_cats=list(num_doc_cl.index)

In [128]:
#li_results

In [122]:
def evaluation_multi_class(li_results,list_cats):
    cat_results_dict = {}
    for cat in list_cats:
        cat_results_dict[cat]=[0,0,0,0]
        for t in li_results:
            if cat==t[1]:
                if cat==t[2]:
                    cat_results_dict[cat][0]+=1
                else:
                    cat_results_dict[cat][1]+=1
            else:
                if cat==t[2]:
                    cat_results_dict[cat][2]+=1
                else:
                    cat_results_dict[cat][3]+=1
    return cat_results_dict

In [123]:
eval_result=evaluation_multi_class(li_results,list_cats)

In [125]:
num_cats_undefined_prec = 0
tot_prec=0
tot_rec=0
a_sum=0
b_sum=0
c_sum=0
d_sum=0
for cat in eval_result:
        a = eval_result[cat][0]
        b = eval_result[cat][1]
        c = eval_result[cat][2]
        d = eval_result[cat][3]
        if a+b==0:
            print ("precision is undefined for category ", cat)
            print ("This category is excluded from precision and recall calculations")
            num_cats_undefined_prec+=1
        else:            
            tot_prec+=a/(a+b)##precision for this category
            tot_rec+=a/(a+c)##recall for this category
            a_sum+=a
            b_sum+=b
            c_sum+=c
            d_sum+=d

In [530]:
macro_precision = tot_prec/(len(eval_result)-num_cats_undefined_prec)
macro_recall = tot_rec/(len(eval_result)-num_cats_undefined_prec)
macro_f_measure = (2*macro_precision*macro_recall)/(macro_precision+macro_recall)

micro_precision = a_sum/(a_sum+b_sum)
micro_recall = a_sum/(a_sum+c_sum)
micro_f_measure = (2*micro_precision*micro_recall)/(micro_precision+micro_recall)

print ("Macro precision=",macro_precision)
print ("Macro recall=",macro_recall)
print ("Macro F-measure=",macro_f_measure)

print ("Micro precision=",micro_precision)
print ("Micro recall=",micro_recall)
print ("Micro F-measure=",micro_f_measure)


Macro precision= 0.5929707286187414
Macro recall= 0.5346135118242998
Macro F-measure= 0.5622820047774385
Micro precision= 0.48756218905472637
Micro recall= 0.5697674418604651
Micro F-measure= 0.5254691689008044
